In [1]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4  # mixtral_moe_7b_instruct_awq
# model_type = ModelType.llama3_70b_instruct_int4

llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    # max_model_len=8192,
    # gpu_memory_utilization=0.95,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)


KeyboardInterrupt: 

In [2]:
generation_config = VllmGenerationConfig(
    max_new_tokens=1024,
    temperature=0,
)

request_list = [
    # {'query': 'Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nINFORMATION:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federal requirements, in effect through at least March 18, include all transportation systems such as airports, planes, trains, and buses. "Preparing to repeal statewide masking mandates at the end of the month is aggressive and optimistic but reasonable," said Dr.\n"Broad mandates are not about individuals. They are put in place to help protect communities, businesses, and healthcare access. Repealing the mask mandate allows people to choose the mitigation layers that are best for them and I have no doubt that many should and will choose to keep mask rules."\nIf these trends continue — and we expect them to —then on Monday, February 28th, we will lift the indoor mask requirement for the State of Illinois," said Governor JB Pritzker. "I want to be clear: Many local jurisdictions, businesses and organizations have their own mask requirements and other mitigations that must be respected.\nSeth Trueger, a Northwestern emergency physician who is also immunocompromised. "Masking has helped slow the spread even in the face of omicron\'s transmissibility. We can and must use this time to further increase vaccination uptake & outreach, especially among children and other populations with low vaccination rates, so when the next wave comes, we will be even better prepared."\nInformation 3:\nPublication date: None\nTitle: Mask and Vaccine Requirements FAQ\'s\nContent:\nOn September 3, 2021, the Governor signed Executive Order 21-22 which requires all individuals over the age of 2 and who can medically tolerate a face covering to wear a face covering when in indoor public places. The Executive Order also requires health care workers, school personnel, higher education personnel and students, and employees and contractors of state-owned or operated congregate facilities to be fully vaccinated, as described in the Order\nInformation 4:\nPublication date: None\nTitle: FAQ for Businesses Concerning Use of Face-Coverings During COVID-19\nContent:\nA: Businesses reserve the right to refuse service to persons unable to comply with the requirement to wear a face covering, but they are required to provide a reasonable accommodation if it does not cause an undue hardship. Businesses are encouraged to inform customers there are exceptions to the requirement that all individuals must wear a mask.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. A: A face covering is a mask or cloth face covering that covers your nose and mouth.\nExceptions may be made for individuals with medical conditions or disabilities that prevent them from safely wearing a face covering. For more information, refer to the questions on reasonable accommodations. A: Masks still must be worn by everyone on planes, buses, trains, and other forms of public transportation; in transportation hubs, such as airports and train and bus stations; in health care settings; and in congregate facilities, such as correctional facilities and homeless shelters.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. When Face Coverings are Required Q: What does it mean to wear a face covering?\nInformation 5:\nPublication date: 2020-04-22\nTitle: Facing Your Face Mask Duties – A List of Statewide Orders | Littler ...\nContent:\n********************************************************NOTE: Given the reduction in activity on this topic,THIS POST WILL NO LONGER BE UPDATED, as of November 10, 2022.********************************************************Governors and public health officials across the country implemented stringent mitigation measures to help contain the spread of COVID-19.\n'},
    
    # {'query': 'Does the conclusion in the answer given below match the ground truth? If it does, please answer "Yes" in the first line; otherwise, answer "No" in the first line. Please provide a reason on the next line.\nGround truth: The claim(Illinois Strengthens Face Mask Rules in Businesses) is true.\nAnswer:True'},
    
    # {'query': 'Below is a claim and some information searched online. These pieces of Information are relevant to the claim. This claim and all information include their respective publication dates and contents. To classify the claim more accurately (if the content described by the claim is correct, it will be classified as true; if the content described by the claim is incorrect, it will be classified as false), please judge the correctness of the claim based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the claim and the date of publication of the information. Just provide the final conclusion without explaining the reasoning process. \n\nClaim:\nPublication date: 2021-01-18\nContent: 13 Israelis suffer FACIAL PARALYSIS after taking Pfizer Covid jab\n\nInformation:\nInformation 1:\nPublication date: 2021-01-17\nTitle: 13 People Develop Facial Paralysis After Receiving Covid-19 Vaccine ...\nContent:\nAccording to WION, the Israeli Health Ministry has indicated that the number of people suffering from such side effects after being inoculated for Covid-19 could be higher. “For at least 28 hours I walked around with it (facial paralysis),” one person told Ynet adding that he recovered after that. This comes just days after 23 people died in Norway after receiving Pfizer’s Covid-19 jab.\nAs many as 13 people in Israel have suffered from facial paralysis after being administered the Covid-19 vaccine, reports claimed. According to WION, the Israeli Health Ministry has indicated that the number of people suffering from such side effects after being inoculated for Covid-19 could be higher.\nOne of the Covid-19 vaccine recipient’s face was paralysed for 28 hours, reports claimed\nInformation 2:\nPublication date: 2021-01-17\nTitle: 13 People in Israel Suffer From Facial Paralysis After Taking ...\nContent:\nJerusalem: At least 13 people in Israel have suffered mild facial paralysis as a side-effect post Coronavirus vaccination, WION reported. The Health Ministry said that the actual number of people suffering from such side effects could be higher than reported. Experts are now apprehensive of giving the second dose of the shot to these people, even though the health ministry has insisted on giving the jab once the paralysis heals.\nExperts are now apprehensive of giving the second dose of the shot to these people, even though the health ministry has insisted on giving the jab once the paralysis heals. “For at least 28 hours I walked around with it (facial paralysis),” one person told Ynet. “I can’t say it was completely gone afterwards, but other than that I had no other pains, except a minor pain where the injection was but there was nothing beyond that.” · Israel began its Covid-19 vaccination drive on December 20, 2020.\nThe Health Ministry said that the actual number of people suffering from such side effects could be higher than reported. Experts are now apprehensive of giving the second dose of the shot to these people, even though the health ministry has insisted on giving the jab once the paralysis heals. “For at least 28 hours I walked around with it (facial paralysis),” one person told Ynet.\n“For at least 28 hours I walked around with it (facial paralysis),” one person told Ynet. “I can’t say it was completely gone afterwards, but other than that I had no other pains, except a minor pain where the injection was but there was nothing beyond that.” · Israel began its Covid-19 vaccination drive on December 20, 2020. About 72 per cent of those aged 60 and over have already been vaccinated. In an similar incident which was reported last month, four volunteers who were given Pfizer vaccine shots during the trial stage had developed Bell’s Palsy in the UK.\nInformation 3:\nPublication date: 2021-01-19\nTitle: 13 Israelis suffer FACIAL PARALYSIS after taking Pfizer Covid jab, ...\nContent:\nAt least 13 Israelis have experienced facial paralysis after being administered the Pfizer Covid-19 vaccine, a month after the US Food and Drug Administration reported similar issues but said they weren’t linked to the jab. ADDITIONAL CONTEXT: Over two million Israelis have been vaccinated against Covid-19 to date, and there has been no evidence the paralysis is linked to the vaccination. The French nonprofit Health Feedback has suggested that the number of facial paralysis cases among the vaccinated “is lower than what would be expected in the general population” and that “incidental illnesses are expected to occur at a certain rate in the general population.”\nFor a handful of Israelis, however, the initiative has led to some unexpected health scares. At least 13 people have reported mild facial paralysis after receiving the Pfizer/BioNTech jab, Israeli outlet Ynet reported, citing the Health Ministry, adding that officials believe the number of such cases could be higher.\nAt least 13 people have reported mild facial paralysis after receiving the Pfizer/BioNTech jab, Israeli outlet Ynet reported, citing the Health Ministry, adding that officials believe the number of such cases could be higher.  · “For at least 28 hours I walked around with it [facial paralysis],” one person who had the side effect told Ynet.\nGalia Rahav, director of the Infectious Diseases Unit at Sheba Medical Center, who said she did not feel “comfortable” with administering the second dose to someone who had received the first jab and subsequently suffered from paralysis.  · “No one knows if this is connected to the vaccine or not. That\'s why I would refrain from giving a second dose to someone who suffered from paralysis after the first dose,” she told the outlet.  · Last month the FDA disclosed that Bell’s palsy, a form of temporary facial paralysis, was reported by four participants during phase three trials of the Pfizer vaccine.\nInformation 4:\nPublication date: None\nTitle: FarsNews Agency 13 Israelis Suffer Facial Paralysis After Taking ...\nContent:\nTEHRAN (FNA)- At least 13 Israelis have experienced facial paralysis after being administered the Pfizer COVID-19 vaccine, a month after the US Food and Drug Administration reported similar issues but said they weren’t linked to the jab. Israel has been hailed for its speedy and efficient mass inoculation program, which has vaccinated a staggering 20 percent of the country’s population since the drive began at the end of December. For a handful of Israelis, however, the initiative has led to some unexpected health scares. At least 13 people have reported mild facial paralysis after receiving the Pfizer/BioNTech jab, Israeli outlet Ynet reported, citing the Health Ministry, adding that officials believe the number of such cases could be higher.\nFor a handful of Israelis, however, the initiative has led to some unexpected health scares. At least 13 people have reported mild facial paralysis after receiving the Pfizer/BioNTech jab, Israeli outlet Ynet reported, citing the Health Ministry, adding that officials believe the number of such cases could be higher.\nAt least 13 people have reported mild facial paralysis after receiving the Pfizer/BioNTech jab, Israeli outlet Ynet reported, citing the Health Ministry, adding that officials believe the number of such cases could be higher. “For at least 28 hours I walked around with it [facial paralysis],” one person who had the side effect told Ynet, adding, “I can\'t say it was completely gone afterwards, but other than that I had no other pains, except a minor pain where the injection was, but there was nothing beyond that.”\nGalia Rahav, director of the Infectious Diseases Unit at Sheba Medical Center, who said she did not feel “comfortable” with administering the second dose to someone who had received the first jab and subsequently suffered from paralysis. “No one knows if this is connected to the vaccine or not. That\'s why I would refrain from giving a second dose to someone who suffered from paralysis after the first dose,” she told the outlet. Last month the FDA disclosed that Bell’s palsy, a form of temporary facial paralysis, was reported by four participants during phase three trials of the Pfizer vaccine.\nInformation 5:\nPublication date: None\nTitle: COVID-19 vaccine: 13 out of 2 mil. Israelis suffer facial paralysis ...\nContent:\nThat\'s why I would refrain from giving a second dose to someone who suffered from paralysis after the first dose." The Health Ministry stated, however, that the second dose should of course be only provided if and when the paralysis passes, Ynet reported. ... More than two million Israelis have received the Pfizer coronavirus vaccine and the majority have experienced no side effects or mild side effects, according to the Health Ministry.\nThe Health Ministry stated, however, that the second dose should of course be only provided if and when the paralysis passes, Ynet reported. ... More than two million Israelis have received the Pfizer coronavirus vaccine and the majority have experienced no side effects or mild side effects, according to the Health Ministry.  · This article was updated and corrected for accuracy on January 20. ... var cont = `Stay Informed As the war against Hamas unfolds, our unwavering newsroom remains committed to covering Israel\'s most profound crisis.\n'}
    {'query': 'who are you?'}

]

resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    use_tqdm=True,
    # verbose=True, prompt_prefix="", output_prefix=""
)

for request, resp in zip(request_list, resp_list):
    print(f"query:\n{request['query']}")
    print()
    print(f"response:\n{resp['response']}")
    break


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:30<00:00, 30.03s/it]

query:
who are you?

response:
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk